In [76]:
import csv
import json
from copy import deepcopy
import pickle as pkl
csv.field_size_limit(1131072)

1131072

## Load Annotation Results

In [80]:
from annotation.analysis.utils.data_util import generate_all_clusters_combine_speakers
from annotation.analysis.utils.data_util import generate_clusters_no_plural_combine_speakers
from annotation.analysis.utils.data_util import collect_mentions
from annotation.adjudication.utils import read_turkle_annotation_multiple_scene, gather_by_scene
from annotation.adjudication.utils import extract_common_cluster, add_to_common_cluster, analyze_types_of_difference, get_disagreement_types, get_error_matrix

In [89]:
# Load Data and relabel the speaker token
data = []
temp = read_turkle_annotation_multiple_scene('source_data/coe_6_7.csv')
data.extend(temp)

# Transform speaker token to string
for item in data:
    temp = []
    for i, (query, answers) in enumerate(item['answers']):
        if isinstance(answers, list):
            for j, answer in enumerate(answers):
                if answer[1]==0 and answer[2]==1 and answer[0]<=(len(item['sentences'])-1):
                    item['answers'][i][1][j] = item['sentences'][answer[0]][0]
                else:
                    item['answers'][i][1][j] = "_".join([str(token) for token in answer])
        else:
            item['answers'][i][1] = [item['answers'][i][1]]
        item['answers'][i][1] = sorted(item['answers'][i][1])

results = []
results.extend(data)
scenes = gather_by_scene(results)

one_way = {}
two_way = {}
for item in scenes:
    if len(scenes[item])==1:
        one_way[item] = scenes[item]
    else:
        two_way[item] = scenes[item]
print(len(two_way), len(one_way))

203 0


In [94]:
# Count Error Matrix
matrix_all = [[] for i in range(7)]
for scene_id in two_way:
    # if scene_id!="s07e17c07t":
    #     continue
    scene = two_way[scene_id]
    answers1, answers2 = scene[0]['answers'], scene[1]['answers']
    labels1, labels2 = get_disagreement_types(answers1, answers2)
    error_matrix = get_error_matrix(labels1, labels2)
    for sent in scene[0]['sentences']:
        print(sent)
    for idx in range(len(error_matrix[0])):
        print(error_matrix[0][idx])
        print(answers1[error_matrix[0][idx]])
        print(answers2[error_matrix[0][idx]])

    print("=="*50)




    for j in range(len(matrix_all)):
        matrix_all[j].extend(error_matrix[j])

['Raj', ':', 'Hey', ',', 'you', 'busy', '?']
['Amy', ':', 'No', '.', 'What', "'s", 'up', '?']
['Raj', ':', 'Have', 'you', 'heard', 'back', 'from', 'Emily', '?']
['Amy', ':', 'I', 'have', '.']
['Raj', ':', 'Great', '!', 'And', '?']
['Amy', ':', 'And', 'I', "'m", 'afraid', 'she', 'does', "n't", 'think', 'you', "'re", 'right', 'for', 'her', '.']
['Raj', ':', 'I', 'give', 'you', 'one', 'simple', 'thing', 'to', 'do--', 'contact', 'a', 'complete', 'stranger', 'and', 'make', 'her', 'fall', 'in', 'love', 'with', 'me--', 'and', 'you', 'blow', 'it', '!']
['Amy', ':', 'I', 'told', 'her', 'what', 'a', 'good', 'guy', 'you', 'are', ',', 'but', 'she', 'thought', 'it', 'was', 'a', 'bad', 'sign', 'that', 'you', 'did', "n't", 'write', 'to', 'her', 'yourself', '.', 'She', 'thought', 'it', 'made', 'you', 'seem', 'too', 'shy', 'and', 'passive', '.']
['Raj', ':', 'I', "'m", 'not', 'too', 'shy', 'and', 'passive', '.', 'You', 'write', 'her', 'back', 'and', 'tell', 'her', 'I', 'said', 'that', '.', 'You', 'know

In [95]:
count = 0
for i, item in enumerate(matrix_all):
    count += len(item)
    print(i, len(item))

0 452
1 310
2 76
3 894
4 293
5 27
6 46


In [99]:
print(count)
print(count * 10 / 60 / 60)

2098
5.827777777777778


In [12]:
matrix_all = [[] for i in range(7)]
for scene_id in two_way:
    # if scene_id!="s07e17c07t":
    #     continue
    scene = two_way[scene_id]
    answers1, answers2 = scene[0]['answers'], scene[1]['answers']
    labels1, labels2 = get_disagreement_types(answers1, answers2)
    error_matrix = get_error_matrix(labels1, labels2)
    for j in range(len(matrix_all)):
        matrix_all[j].extend(error_matrix[j])


In [6]:
for scene_id in two_way:
    if scene_id!="s07e17c07t":
        continue
    scene = two_way[scene_id]
    answers1, answers2 = scene[0]['answers'], scene[1]['answers']
    labels1, labels2 = get_disagreement_types(answers1, answers2)
    error_matrix = get_error_matrix(labels1, labels2)

    for i, (a1, a2, l1, l2) in enumerate(zip(answers1, answers2, labels1, labels2)):
        if l1==l2:
            continue
        print(a1, l1)
        print(a2, l2)
        print()

[(0, 4, 5), ['notPresent']] -2
[(0, 4, 5), ['Amy']] 0

[(1, 4, 5), ['notMention']] -3
[(1, 4, 5), ['notPresent']] -2

[(6, 5, 22), ['notMention']] -3
[(6, 5, 22), ['notPresent']] -2

[(6, 19, 20), ['notMention']] -3
[(6, 19, 20), ['notPresent']] -2

[(6, 25, 26), ['notMention']] -3
[(6, 25, 26), ['6_5_22']] -1

[(7, 5, 6), ['notMention']] -3
[(7, 5, 6), ['notPresent']] -2

[(7, 31, 32), ['notMention']] -3
[(7, 31, 32), ['notPresent']] -2

[(8, 19, 20), ['notMention']] -3
[(8, 19, 20), ['notPresent']] -2

[(8, 27, 29), ['notMention']] -3
[(8, 27, 29), ['notPresent']] -2

[(9, 13, 15), ['notMention']] -3
[(9, 13, 15), ['notPresent']] -2

[(10, 2, 3), ['notMention']] -3
[(10, 2, 3), ['notPresent']] -2

[(10, 5, 7), ['notMention']] -3
[(10, 5, 7), ['notPresent']] -2

[(12, 2, 3), ['notMention']] -3
[(12, 2, 3), ['notPresent']] -2

[(13, 5, 6), ['2_7_8', 'Amy']] 6
[(13, 5, 6), ['11_5_6', 'Amy']] -1

[(13, 8, 9), ['2_7_8', 'Amy']] 6
[(13, 8, 9), ['11_5_6', 'Amy']] -1

[(13, 15, 17), ['notMen

In [4]:
all_num = 0
disagreed_num = 0
for item in two_way:
    if item!="s07e17c07t":
        continue
    scene = two_way[item]
    # Step 1: Build a common cluster with exact match
    common_clusters = extract_common_cluster(scene[0]['answers'], scene[1]['answers'])
    # Step 2: Incrementally add [query, annotation] belonging to the same cluster
    complete_clusters = add_to_common_cluster(common_clusters, scene[0]['answers'], scene[1]['answers'])
    # Step 3: assign label to each (query, annotation) and decide whether they disagree
    # a, b, c = analyze_difference(complete_clusters, scene[0]['answers'], scene[1]['answers'])
    # all_num += a
    # disagreed_num += (a-b)

    a, b, c = analyze_types_of_difference(complete_clusters, scene[0]['answers'], scene[1]['answers'])
    all_num += a
    disagreed_num += b
    # print(item, a, b, a-b, str(c)+"%")
print(all_num, disagreed_num, disagreed_num/all_num*100)

78 14 17.94871794871795
